You have two DataFrames:
df_sales: sale_id, product_id, sale_date, amount
df_products: product_id, product_name, category
Write PySpark code to:

Join the two DataFrames on product_id
Filter for sales from the last 30 days (assume today's date using current_date())
Calculate the total sales amount per category
Find the top 3 categories by total sales
Include a column showing each category's percentage of total sales

In [ ]:
%python

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType
from pyspark.sql.functions import col, date_sub, broadcast, current_date, round, sum as agg_sum, lit


spark = SparkSession.builder.appName("practice_questions").getOrCreate()

sales_schema = StructType([
    StructField("sale_id", StringType()),
    StructField("product_id", StringType()),
    StructField("sale_date", DateType()),
    StructField("amount", FloatType())
])

products_schema = StructType([
    StructField("product_id", StringType()),
    StructField("product_name", StringType()),
    StructField("category", StringType())
])

sales_df = spark.read.format("csv").schema(sales_schema).option("header", "True").load("dfbs:/raw/data/sales.csv")
products_df = spark.read.format("parquet").schema(products_df).load("dbfs:/raw/data/products.parquet")

sales_df = sales_df.filter(col("sale_date") < date_sub(current_date(), -30)) # filter early for predicate pushdown
joined_df = sales_df.join(broadcast(products_df), "product_id", "inner")
joined_df = joined_df.select("category", "amount") # predicate pushdown

joined_df = joined_df.groupBy("category").agg(agg_sum("amount").alias("total_sales"))
total_sale_sum = joined_df.agg(agg_sum("total_sales")).collect()[0][0]
window_spec = Window.orderBy(col("total_sales").desc())


ranked_df = joined_df.withColumn("percentage_sum", round(col("total_sales") * 100 / lit(total_sale_sum) , 2))
ranked_df = joined_df.dense_rank().over(window_spec).alias("category_rank")
top_df = ranked_df.filter(col("category_rank") <= 3)
top_df.show()

%md

You have a DataFrame: employee_id, department, salary, hire_date
Write PySpark code to:

For each department, assign a rank to employees based on salary (highest salary = rank 1)
Add a column showing the salary difference between each employee and the next highest-paid employee in their department
Add a column showing each employee's salary percentile within their department (use percent_rank)
Filter to show only employees in the top 2 ranks per department

In [ ]:
from pyspark.sql import  SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DateType, StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, percent_rank, lead
spark  = SparkSession.builder.appName("practice").getOrCreate()
schema = StructType(
    [
        StructField("employee_id", StringType()),
        StructField("department", StringType()),
        StructField("salary", FloatType()),
        StructField("hire_date", DateType()),

    ]
)
df = spark.read.format("csv").option("header", "True").schema(schema).load("dbfs:/raw/data/employee.csv")

department_spec = Window.partitionBy("department").orderBy(col("salary").desc())
df = df.dense_rank().over(department_spec).alias("salary_rank")
df = df.withColumn("salary_diff",col("salary") - lead("salary", 1).over(department_spec))
df = df.withColumn("department_percentile_rank", percent_rank().over(department_spec))
df = df.filter(col("salary_rank") <= 2).orderBy("department", "salary_rank")
df.show()

You have a DataFrame: transaction_id, user_id, transaction_date, amount, transaction_type (values: 'credit' or 'debit')

Write PySpark code to:

Calculate running balance for each user (credits add, debits subtract) ordered by transaction_date
Identify users who had negative balance at any point
For these users, calculate: total transactions, total credits, total debits, and minimum balance reached
Sort by minimum balance (most negative first)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, FloatType, StringType
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum as agg_sum, when, count, min, lit

spark = SparkSession.builder.appName("practice").getOrCreate()
schema = StructType([
    StructField("transaction_id", StringType()),
    StructField("user_id", StringType()),
    StructField("transaction_date", DateType()),
    StructField("amount", FloatType()),
    StructField("transaction_type", StringType()),
])

df = spark.read.format("delta").schema(schema).load("transactions")
window_spec = Window.partitionBy("user_id").orderBy("transaction_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df = df.withColumn("signed_amount", 
                   when(col("transaction_type") == "credit", col("amount"))
                   .otherwise(-col("amount")))
df = df.withColumn("running_balance", agg_sum("signed_amount").over(window_spec))
# Find users with negative balance at any point
negative_balance_users = df.filter(col("running_balance") < 0).select("user_id").distinct()

# Filter to only those users
df= df.join(negative_balance_users, "user_id", "inner")

df = df.groupBy("user_id").agg(count("*").alias("total_transactions"), agg_sum(when(col("transaction_type") == "credit", col("amount")).otherwise(lit(0))).alias("total_credits"), agg_sum(when(col("transaction_type") == "debit", col("amount")).otherwise(lit(0))).alias("total_debits"), min("running_balance").alias("minimum_balance"))
df = df.orderBy(col("minimum_balance")).show()


You have a DataFrame with columns: product_id, category, price, stock_quantity
Write PySpark code to:

Find products where stock_quantity is null and replace with 0
Create a new column stock_status that shows:

"Out of Stock" if stock_quantity = 0
"Low Stock" if stock_quantity < 10
"In Stock" if stock_quantity >= 10


Filter for products in "Electronics" or "Computers" category
Show results ordered by price descending

In [ ]:
df = df.fillna(0, subset=["stock_quantity"]).withColumn("stock_status", when(col("stock_quantity") == 0, "Out of Stock").otherwise(when(col("stock_quantity") < 10, "Low Stock").otherwise("In Stock")))
df = df.filter(col("product_id") == "Electronics" | col("product_id") == "Computers").orderBy(col("price").desc())
df.show()

orders_df: order_id, customer_id, order_date, total_amount
customers_df: customer_id, customer_name, city, registration_date
Write PySpark code to:

Perform a left join to get all orders with customer details
Find customers who have placed more than 3 orders
For these customers, calculate their average order amount
Show customer_name, city, order_count, and avg_order_amount
Sort by order_count descending

In [ ]:
joined_df = orders_df.join(customers_df, "customer_id", "left")
joined_df = joined_df.groupBy("customer_id", "customer_name", "city").agg(count("*").alias("order_count"), round(avg("total_amount"), 2).alias("avg_order_amount"))
joined_df = joined_df.filter(col("order_count") > 3).orderBy(col("order_count").desc()).select("customer_name", "city", "avg_order_amount")
joined_df.show()

Write PySpark code to:

Calculate 7-day moving average of sales_amount for each product-region combination
Pivot the data to show regions as columns with their moving averages
Filter to show only the last 30 days of data (use datediff and current_date)
Round moving averages to 2 decimal places